In [1]:
import os
import sys

# Obtener el directorio actual de la notebook
notebook_dir = os.getcwd()

# Obtener el directorio padre
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Agregar el directorio padre al path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from libs.config.config_variables import (
    MAX_WORKERS,
    TIMEOUT_API_REQUEST,
    CATALOG_CSV_DIR,
)

from modules.catalog_scraper.seismic_scraper import SeismicScraper
from modules.downloader.records_downloader import SeismicDownloader

# 0. Configuración básica de catálogos

In [3]:
# Límites geográficos (Perú y alrededores)
MIN_LAT, MAX_LAT = -20.0, 0.0
MIN_LON, MAX_LON = -85.0, -68.0

# Año de inicio del scraping
START_YEAR = 1400

# 1. Ejecución de scraping de catálogos


## 1.1 Carga de datos históricos

In [4]:
# Registrar los catálogos csv
historical_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST, csv_base_path=CATALOG_CSV_DIR
)
results = historical_scraper.scrape_events_from_csv(
    catalogs=["gcmt", "isc-gem", "sara"],
    # start_year=START_YEAR,
    # min_lat=MIN_LAT,
    # max_lat=MAX_LAT,
    # min_lon=MIN_LON,
    # max_lon=MAX_LON,
)


2025-11-23 15:47:20,203 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events_from_csv
2025-11-23 15:47:20,207 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 1 archivos CSV para GCMT
2025-11-23 15:47:20,210 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para ISC-GEM
2025-11-23 15:47:20,213 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para SARA
2025-11-23 15:47:20,216 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: 5 archivos con 40 workers
2025-11-23 15:47:20,220 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: GCMT - gcmt_data.csv
2025-11-23 15:47:20,223 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-cat.csv
2025-11-23 15:47:20,254 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-suppl.csv
2025-11-23 15:47:20,254 [INFO] modules.ca

In [5]:
historical_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|     178748      | 178748 |      0       |    1    |
+-----------------+--------+--------------+---------+


## 1.2 Actualización de datos recientes

In [6]:
recent_data_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST
)

results = recent_data_scraper.scrape_events(
    start_year=2000,
    min_lat=MIN_LAT,
    max_lat=MAX_LAT,
    min_lon=MIN_LON,
    max_lon=MAX_LON,
    catalogs=["usgs", "isc", "igp"],
)

2025-11-23 15:47:50,247 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events
2025-11-23 15:47:50,255 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: 78 tareas con 40 workers
2025-11-23 15:47:50,255 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2000
2025-11-23 15:47:50,280 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2001
2025-11-23 15:47:50,280 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2002
2025-11-23 15:47:50,280 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2003
2025-11-23 15:47:50,307 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2004
2025-11-23 15:47:50,338 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2005
2025-11-23 15:47:50,338 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2006
2025-11-23 15:47:50,349 [INFO] m

In [7]:
recent_data_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|      16067      | 15978  |      89      |   15    |
+-----------------+--------+--------------+---------+


## 1.3 Descarga de registros acelerométricos

In [8]:
NUM_EVENTS = 100
PARALLEL_STATIONS = 10
CATALOG_PARSER_CONFIG = {
    "igp": 200,
    # "usgs": 5,
    # "emsc": 15,
}

In [9]:
downloader = SeismicDownloader()

downloader.process_multiple_catalogs(
    catalogs=CATALOG_PARSER_CONFIG,
    max_workers=MAX_WORKERS,
    parallel_stations=PARALLEL_STATIONS,
)

2025-11-23 15:48:09,862 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_multiple_catalogs
2025-11-23 15:48:09,862 [INFO] modules.downloader.records_downloader: Iniciando procesamiento de 1 catálogos
2025-11-23 15:48:09,862 [INFO] modules.downloader.records_downloader: Procesando catálogo: IGP (200 eventos)
2025-11-23 15:48:09,871 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_events
2025-11-23 15:48:09,871 [INFO] modules.downloader.records_downloader: Eventos a procesar en IGP: 200
2025-11-23 15:48:09,871 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_catalog
2025-11-23 15:48:09,879 [INFO] modules.downloader.records_downloader: Iniciando procesamiento: IGP - 200 eventos
2025-11-23 15:48:09,879 [INFO] libs.database.seismic_records: Inicio de sesión de sincronización (record_id=1, catalog_id=123)
2025-11-23 15:48:09,887 [WARNING] modules.downloader.r

{'success': True,
 'catalogs_processed': 1,
 'results': {'igp': {'result': {'success': True,
    'catalog': 'igp',
    'sync_id': 1,
    'status': 'completed_with_errors',
    'total_events': 200,
    'processed_events': 200,
    'successful_events': 0,
    'failed_events': 200,
    'total_stations': 0,
    'total_samples': 0,
    'processing_time': 1158.3836688995361,
    'errors': ['2025-0020: Procesamiento fallido',
     '2025-0004: Procesamiento fallido',
     '2025-0016: Procesamiento fallido',
     '2025-0035: Procesamiento fallido',
     '2025-0013: Procesamiento fallido',
     '2025-0034: Procesamiento fallido',
     '2025-0027: Procesamiento fallido',
     '2025-0011: Procesamiento fallido',
     '2025-0029: Procesamiento fallido',
     '2025-0041: Procesamiento fallido']},
   'total_events': 200,
   'processed_events': 200,
   'successful_events': 0,
   'failed_events': 200,
   'total_stations': 0,
   'total_samples': 0,
   'processing_time': 1158.3836688995361}}}

In [10]:
downloader.metadata

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                                                                                         igp                                                  